<a href="https://colab.research.google.com/github/ayushman1309/ML_LAB/blob/main/Linear%26Lasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
import os
from textwrap import dedent

In [2]:
X = np.array([[1.0], [2.0], [3.0], [4.0], [5.0]])   # shape (n, d)
y = np.array([1.2, 1.9, 3.2, 3.9, 5.1])            # shape (n,)

In [3]:
ridge_alpha = 1.0    # regularization strength for Ridge
lasso_alpha = 0.1    # regularization strength for Lasso

In [5]:
lin = LinearRegression(fit_intercept=True).fit(X, y)
rid = Ridge(alpha=ridge_alpha, fit_intercept=True).fit(X, y)
las = Lasso(alpha=lasso_alpha, fit_intercept=True, max_iter=10000, tol=1e-6).fit(X, y)

In [6]:
models = {
    "Linear": lin,
    "Ridge": rid,
    "Lasso": las
}

In [7]:
def manual_prediction_lines(intercept, coefs, x_row, idx=None, precision=6):
    intercept_val = float(intercept) if intercept is not None else 0.0
    coefs = np.atleast_1d(coefs)
    x_row = np.atleast_1d(x_row)
    terms = []
    for j, (c, xv) in enumerate(zip(coefs, x_row)):
        terms.append(f"({c:+.{precision}f} * {xv:.{precision}f})")
    expr = " + ".join(terms)
    if intercept is not None:
        full_expr = f"{intercept_val:+.{precision}f} + {expr}" if expr else f"{intercept_val:+.{precision}f}"
    else:
        full_expr = expr
    pred_val = intercept_val + float(np.dot(coefs, x_row))
    idx_str = f"Point {idx}: " if idx is not None else ""
    line = f"{idx_str}X={list(x_row)}, prediction = {full_expr} = {pred_val:.{precision}f}"
    return pred_val, line

In [8]:
obs_lines = []
obs_lines.append("Regression observation book\n")
obs_lines.append("Dataset (first lines):")
for i, (xi, yi) in enumerate(zip(X, y)):
    obs_lines.append(f"  Point {i+1}: X = {list(np.atleast_1d(xi))}, y(actual) = {yi:.6f}")

results = {}
for name, model in models.items():
    intercept = float(model.intercept_)
    coefs = np.atleast_1d(model.coef_)
    preds = []
    lines = []
    obs_lines.append("\n" + "-"*60)
    obs_lines.append(f"{name} Regression (alpha = {'N/A' if name=='Linear' else (ridge_alpha if name=='Ridge' else lasso_alpha)})")
    obs_lines.append(f"Intercept = {intercept:.6f}")
    obs_lines.append("Coefficient(s) = [" + ", ".join(f"{c:.6f}" for c in coefs) + "]")
    obs_lines.append("Manual per-point predictions (showing arithmetic):")
    for i, x_row in enumerate(X, start=1):
        p, l = manual_prediction_lines(intercept, coefs, x_row, idx=i, precision=6)
        preds.append(p)
        obs_lines.append("  " + l + f", actual = {y[i-1]:.6f}")
    mse_val = mean_squared_error(y, np.array(preds))
    obs_lines.append(f"{name} MSE = {mse_val:.6f}")
    results[name] = {"mse": mse_val, "preds": preds, "lines": lines}

In [9]:
obs_lines.append("\n" + "="*60)
obs_lines.append("MSE Comparison:")
for name, info in results.items():
    obs_lines.append(f"  {name}: MSE = {info['mse']:.6f}")
best = min(results.keys(), key=lambda k: results[k]['mse'])
obs_lines.append(f"\nBest method on this dataset (lowest MSE): {best} (MSE = {results[best]['mse']:.6f})")

In [10]:
os.makedirs("/mnt/data", exist_ok=True)   # Colab persistent workspace
obs_path = "/mnt/data/regression_observation_book.txt"
with open(obs_path, "w") as f:
    f.write("\n".join(obs_lines))

In [11]:
print("\n".join(obs_lines[:50]))  # print first 50 lines to avoid huge output - modify as needed
print("\nObservation book saved to:", obs_path)
print("\nFull MSEs:")
for name, info in results.items():
    print(f"  {name}: MSE = {info['mse']:.6f}")

Regression observation book

Dataset (first lines):
  Point 1: X = [np.float64(1.0)], y(actual) = 1.200000
  Point 2: X = [np.float64(2.0)], y(actual) = 1.900000
  Point 3: X = [np.float64(3.0)], y(actual) = 3.200000
  Point 4: X = [np.float64(4.0)], y(actual) = 3.900000
  Point 5: X = [np.float64(5.0)], y(actual) = 5.100000

------------------------------------------------------------
Linear Regression (alpha = N/A)
Intercept = 0.120000
Coefficient(s) = [0.980000]
Manual per-point predictions (showing arithmetic):
  Point 1: X=[np.float64(1.0)], prediction = +0.120000 + (+0.980000 * 1.000000) = 1.100000, actual = 1.200000
  Point 2: X=[np.float64(2.0)], prediction = +0.120000 + (+0.980000 * 2.000000) = 2.080000, actual = 1.900000
  Point 3: X=[np.float64(3.0)], prediction = +0.120000 + (+0.980000 * 3.000000) = 3.060000, actual = 3.200000
  Point 4: X=[np.float64(4.0)], prediction = +0.120000 + (+0.980000 * 4.000000) = 4.040000, actual = 3.900000
  Point 5: X=[np.float64(5.0)], predict